In [11]:
import pandas as pd
import pymysql
import os
import json
import base64
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [29]:
df = pd.read_excel('daily.xls', header=2)
df = df[['제조수입사\n관리번호', '신청\n번호', "사회계층\n유형", "생애최초 차량\n구매자 여부"]]
df = df[df['제조수입사\n관리번호'].notna()]  # '제조수입사\n관리번호' NaN 값 행 제외
df = df.rename(columns={'제조수입사\n관리번호': 'RN', '생애최초 차량\n구매자 여부': '청년생애'})
df['신청\n번호'] = pd.to_numeric(df['신청\n번호'], errors='coerce').astype('Int64')  # int형으로 표시
df.head()

,RN,신청\n번호,사회계층\n유형,청년생애
0,RN125048018,95,다자녀가구(2자녀),Y
1,RN124337477,91,NaN,Y
2,RN124390999,90,NaN,N


In [30]:
DB_CONFIG = {
    'host': '192.168.0.114',
    'port': 3306,
    'user': 'my_pc_user',
    'password': '!Qdhdbrclf56',
    'db': 'greetlounge',
    'charset': 'utf8mb4'
}

# DF의 RN에 맞는 subsidy_applications.recent_thread_id를 머지하여 DF에 붙임
conn = pymysql.connect(**DB_CONFIG)

# RN 목록을 문자열로 쿼리용 포맷 생성
rn_list = df['RN'].dropna().unique().tolist()
if rn_list:
    # RN 모두 str로 처리
    rn_str_list = [str(rn) for rn in rn_list]
    # 쿼리에 넣을 포맷 ("'RN1','RN2',...")
    rn_tuple_sql = ",".join([f"'{rn}'" for rn in rn_str_list])
else:
    rn_tuple_sql = "''"  # 빈 리스트 방지

sql = f"""
SELECT 
    RN, recent_thread_id
FROM
    subsidy_applications
WHERE
    RN IN ({rn_tuple_sql})
"""

df_threads = pd.read_sql(sql, conn)
df = pd.merge(df, df_threads, on='RN', how='left')

conn.close()


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_6016\4169080103.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_threads = pd.read_sql(sql, conn)


In [31]:
df.head()

,RN,신청\n번호,사회계층\n유형,청년생애,recent_thread_id
0,RN125048018,95,다자녀가구(2자녀),Y,19ab32653891313a
1,RN124337477,91,NaN,Y,19aa55250a61506d
2,RN124390999,90,NaN,N,19ab3050e320d137


In [ ]:
import os
import base64
import re
import time
import pandas as pd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 토큰 파일 경로 설정 (노트북 환경에 맞게 조정)
TOKEN_FILE = 'token123.json'
CREDENTIALS_FILE = 'credentials_3.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    """Gmail API 서비스 객체 생성"""
    creds = None
    if os.path.exists(TOKEN_FILE):
        try:
            creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
        except Exception:
            pass

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
            except Exception:
                creds = None
        
        if not creds:
            print("❌ 유효한 토큰이 없으며 자동 갱신도 실패했습니다.")
            return None
            
    return build('gmail', 'v1', credentials=creds)

def extract_emails(text):
    """텍스트에서 이메일 주소 추출"""
    if not text:
        return []
    return re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', text)

def create_auto_reply_content(row):
    """DataFrame 행 기반 자동 답장 본문 생성"""
    rn = row.get('RN', '')
    app_num = row.get('신청\n번호', '')
    
    # 사회계층 유형 가져오기
    social_val = row.get('사회계층\n유형', '')
    youth_val = row.get('청년생애', '')
    
    types = []
    
    # 1. 사회계층 유형 체크
    if not pd.isna(social_val) and str(social_val).strip():
        types.append(str(social_val).strip())
        
    # 2. 청년생애 체크 (값이 'Y'인 경우)
    if not pd.isna(youth_val) and str(youth_val).strip().upper() == 'Y':
        types.append('청년생애')
        
    # 슬래시(/)로 연결
    final_type_str = " / ".join(types)
    
    # 기본 내용 구성
    lines = [
        "안녕하세요.",
        "",
        f"- RN: {rn}",
        f"- 신청번호: {app_num}"
    ]
    
    # 병합된 유형 정보가 있으면 줄 추가
    if final_type_str:
        lines.append(f"- 특이사항: {final_type_str}")
    
    # 마무리 인사 추가
    lines.extend([
        "",
        "위 정보로 접수되었습니다.",
        "감사합니다."
    ])
    
    return "\n".join(lines)

def debug_reply_all_batch(df):
    """
    [DEBUG 모드] 실제 전송 없이 전송될 내용을 출력합니다.
    """
    service = get_gmail_service()
    if not service:
        print("Gmail 서비스 연결 실패 - 로컬 토큰을 확인해주세요.")
        return

    print(f"🚀 [DEBUG] 총 {len(df)}건의 메일 전송 시뮬레이션을 시작합니다.\n")
    
    for idx, row in df.iterrows():
        rn = row.get('RN', f'Unknown-{idx}')
        thread_id = row.get('recent_thread_id')
        
        print(f"--- [Case {idx+1}] RN: {rn} ---")
        
        if not thread_id:
            print(f"⚠️ [Skip] thread_id가 없습니다.")
            print("-" * 40)
            continue

        try:
            # 1. 원본 스레드 조회 (실제 API 호출은 필요함 - 헤더 정보를 얻기 위해)
            print(f"🔍 스레드 조회 중... (ID: {thread_id})")
            thread = service.users().threads().get(userId='me', id=thread_id).execute()
            messages = thread.get('messages', [])
            
            if not messages:
                print(f"⚠️ [Skip] 메시지가 없습니다.")
                print("-" * 40)
                continue
                
            last_msg = messages[-1]
            msg_detail = service.users().messages().get(userId='me', id=last_msg['id'], format='full').execute()
            headers = msg_detail['payload']['headers']
            
            # 2. 헤더 파싱
            subject = next((h['value'] for h in headers if h['name'].lower() == 'subject'), '(제목 없음)')
            from_header = next((h['value'] for h in headers if h['name'].lower() == 'from'), '')
            cc_header = next((h['value'] for h in headers if h['name'].lower() == 'cc'), '')
            to_header = next((h['value'] for h in headers if h['name'].lower() == 'to'), '')
            
            # 3. 수신자 설정 시뮬레이션
            to_emails = extract_emails(from_header)
            cc_emails_raw = extract_emails(cc_header) + extract_emails(to_header)
            cc_emails = list(set(cc_emails_raw) - set(to_emails))
            
            # 4. 본문 및 제목 생성
            reply_content = create_auto_reply_content(row)
            new_subject = subject if subject.strip().lower().startswith('re:') else f"Re: {subject}"
            
            # --- 디버그 출력 ---
            print(f"✅ [전송 예정 정보]")
            print(f"   - Thread ID: {thread_id}")
            print(f"   - 제목: {new_subject}")
            print(f"   - To (발신자): {to_emails}")
            print(f"   - Cc (참조+수신자): {cc_emails}")
            print(f"   - 본문 미리보기:\n{'-'*20}\n{reply_content}\n{'-'*20}")
            
            # 실제 전송 코드는 주석 처리됨
            # service.users().messages().send(...).execute()
            print(f"👉 (API 전송 호출은 건너뛰었습니다)")
            
        except Exception as e:
            print(f"❌ [Error] {e}")
        
        print("-" * 40 + "\n")

# 테스트 실행
debug_reply_all_batch(df)

🚀 [DEBUG] 총 3건의 메일 전송 시뮬레이션을 시작합니다.

--- [Case 1] RN: RN125048018 ---
🔍 스레드 조회 중... (ID: 19ab32653891313a)
✅ [전송 예정 정보]
   - Thread ID: 19ab32653891313a
   - 제목: Re: [Null] 보조금신청 RN125048018 / 영암군 / Model Y R / 오종택 다자녀2 / 
   - To (발신자): ['tesla003@greetlounge.com']
   - Cc (참조+수신자): ['minjpark@tesla.com', 'eryoo@tesla.com']
   - 본문 미리보기:
--------------------
안녕하세요.

- RN: RN125048018
- 신청번호: 95
- 특이사항: 다자녀가구(2자녀) / 청년생애

위 정보로 정상 접수되었습니다.
감사합니다.
--------------------
👉 (API 전송 호출은 건너뛰었습니다)
----------------------------------------

--- [Case 2] RN: RN124337477 ---
🔍 스레드 조회 중... (ID: 19aa55250a61506d)
✅ [전송 예정 정보]
   - Thread ID: 19aa55250a61506d
   - 제목: Re: [NULL] 보조금신청 RN124337477 / 영암군 / Model Y L  / 강명진 / 
   - To (발신자): ['tesla003@greetlounge.com']
   - Cc (참조+수신자): ['woojukim@tesla.com', 'minjpark@tesla.com']
   - 본문 미리보기:
--------------------
안녕하세요.

- RN: RN124337477
- 신청번호: 91
- 특이사항: 청년생애

위 정보로 정상 접수되었습니다.
감사합니다.
--------------------
👉 (API 전송 호출은 건너뛰었습니다)
-------------------

In [ ]:
import os
import base64
import re
import time
import pandas as pd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# ---------------------------------------------------------
# 환경 설정
# ---------------------------------------------------------
TOKEN_FILE = 'token123.json'
CREDENTIALS_FILE = 'credentials_3.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    """Gmail API 서비스 객체 생성"""
    creds = None
    if os.path.exists(TOKEN_FILE):
        try:
            creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
        except Exception:
            pass

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
            except Exception:
                creds = None
        
        if not creds:
            print("❌ 유효한 토큰이 없으며 자동 갱신도 실패했습니다.")
            return None
            
    return build('gmail', 'v1', credentials=creds)

def extract_emails(text):
    """텍스트에서 이메일 주소 추출"""
    if not text:
        return []
    return re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', text)

def create_auto_reply_content(row):
    """DataFrame 행 기반 자동 답장 본문 생성"""
    rn = row.get('RN', '')
    app_num = row.get('신청\n번호', '')
    social_type = row.get('사회계층\n유형', '')
    
    # 사회계층 유형이 없으면(NaN 등) 빈 문자열 처리
    if pd.isna(social_type):
        social_type = '-'

    content = f"""안녕하세요.

- RN: {rn}
- 신청번호: {app_num}
- 우선순위: {social_type}

위 정보로 정상 접수되었습니다.
감사합니다."""
    return content

def send_reply_all_batch(df):
    """
    DataFrame을 순회하며 실제 '전체 답장'을 발송합니다.
    """
    service = get_gmail_service()
    if not service:
        print("Gmail 서비스 연결 실패 - 로컬 토큰을 확인해주세요.")
        return

    print(f"🚀 총 {len(df)}건의 메일 발송을 시작합니다.\n")
    
    success_count = 0
    fail_count = 0

    for idx, row in df.iterrows():
        rn = row.get('RN', f'Unknown-{idx}')
        thread_id = row.get('recent_thread_id')
        
        print(f"--- [진행중 {idx+1}/{len(df)}] RN: {rn} ---")
        
        if not thread_id or pd.isna(thread_id):
            print(f"⚠️ [Skip] thread_id가 없습니다.")
            fail_count += 1
            print("-" * 40)
            continue

        try:
            # 1. 원본 스레드의 최신 메시지 조회
            thread = service.users().threads().get(userId='me', id=thread_id).execute()
            messages = thread.get('messages', [])
            
            if not messages:
                print(f"⚠️ [Skip] 메일 메시지를 찾을 수 없습니다.")
                fail_count += 1
                continue
                
            last_msg = messages[-1]
            msg_detail = service.users().messages().get(userId='me', id=last_msg['id'], format='full').execute()
            headers = msg_detail['payload']['headers']
            
            # 2. 헤더 파싱
            subject = next((h['value'] for h in headers if h['name'].lower() == 'subject'), '(제목 없음)')
            from_header = next((h['value'] for h in headers if h['name'].lower() == 'from'), '')
            cc_header = next((h['value'] for h in headers if h['name'].lower() == 'cc'), '')
            to_header = next((h['value'] for h in headers if h['name'].lower() == 'to'), '')
            message_id_header = next((h['value'] for h in headers if h['name'].lower() == 'message-id'), '')
            references_header = next((h['value'] for h in headers if h['name'].lower() == 'references'), '')
            
            # 3. 수신자 설정 (전체 답장)
            # - To: 원본 발신자
            to_emails = extract_emails(from_header)
            
            # - Cc: (원본 Cc + 원본 To) - (이번 답장의 To)
            cc_emails_raw = extract_emails(cc_header) + extract_emails(to_header)
            
            # 내 이메일 주소 가져오기 (서비스 프로필 정보 이용)
            my_profile = service.users().getProfile(userId='me').execute()
            my_email = my_profile.get('emailAddress', '')
            
            # 필터링: (To에 있는 사람 제외) AND (내 이메일 제외)
            # set() 연산 후 다시 list로 변환하여 중복 제거
            exclude_emails = set(to_emails)
            if my_email:
                exclude_emails.add(my_email)
                
            cc_emails = list(set(cc_emails_raw) - exclude_emails)
            
            # 4. 본문 및 메시지 구성
            reply_content = create_auto_reply_content(row)
            
            message = MIMEMultipart()
            message['to'] = ', '.join(to_emails)
            if cc_emails:
                message['cc'] = ', '.join(cc_emails)
            
            # 제목: Re: 가 없으면 추가
            new_subject = subject if subject.strip().lower().startswith('re:') else f"Re: {subject}"
            message['subject'] = new_subject
            
            # 5. 스레드 체이닝 (답장 연결)
            if message_id_header:
                message['In-Reply-To'] = message_id_header
                # References 이어붙이기
                new_references = f"{references_header} {message_id_header}".strip()
                message['References'] = new_references
            
            # 본문 첨부
            message.attach(MIMEText(reply_content, 'plain'))
            
            # 6. 실제 전송
            raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')
            service.users().messages().send(
                userId='me',
                body={'raw': raw_message, 'threadId': thread_id}
            ).execute()
            
            print(f"✅ [발송 성공] Thread ID: {thread_id}")
            success_count += 1
            
            # API 부하 방지를 위한 대기 (1초)
            time.sleep(1)
            
        except Exception as e:
            print(f"❌ [발송 실패] {e}")
            fail_count += 1
        
        print("-" * 40 + "\n")

    print(f"🎉 작업 종료: 성공 {success_count}건, 실패 {fail_count}건")

# 실제 실행 시 주석 해제
send_reply_all_batch(df)

🚀 총 3건의 메일 발송을 시작합니다.

--- [진행중 1/3] RN: RN125048018 ---
✅ [발송 성공] Thread ID: 19ab32653891313a
----------------------------------------

--- [진행중 2/3] RN: RN124337477 ---
✅ [발송 성공] Thread ID: 19aa55250a61506d
----------------------------------------

--- [진행중 3/3] RN: RN124390999 ---
✅ [발송 성공] Thread ID: 19ab3050e320d137
----------------------------------------

🎉 작업 종료: 성공 3건, 실패 0건
